## New IMDb Keyword Search
Input a list of keywords into a scraper to get a dataset of movies on IMDb with any of those keywords.

To run a smaller retrieval (max 50 items per keyword), set `is_test` to True. 

For an even smaller test (17 items) I like to run "scary-clowns" as the keyword.

Make sure you run the MovieScraper class before you start! :)

> _**NOTE:** If you get an `IMDbDataAccessError`, you've hit the rate limit and IMDb has blocked you from retrieving more data._

In [28]:
# ADD KEYWORDS - Multiple words must have dashes instead of spaces
my_keywords = ["remake", "reboot", 
               "based-on-novel", "based-on-book", "based-on-play", 
               "based-on-comic", "based-on-manga", "based-on-film", 
               "based-on-tv-series", "based-on-comic-book", "based-on-short-story"]
test_keywords = ["reboot", "no-background-score"]
scraper = MovieScraper(keywords = my_keywords, is_test = False)

BASED-ON-SHORT-STORY - Collected 2278 IDs over 47 pages
Total: 38417 IDs collected - Removed 5046 duplicates


### Preserving 5-24 pulls
They are incomplete because IMDb has a rate limit at 200 pages. Will try again tomorrow with get_more_pages function.

In [42]:
# Write the list to a text file
with open("incomplete_scraper_ids_5-24.txt", "w") as file:
    file.write('\n'.join(scraper.ids))

# Read the list from the text file
with open("incomplete_scraper_ids_5-24.txt", "r") as file:
    loaded_list = file.read().splitlines()

print(loaded_list)

['0209746', '0027098', '0372183', '1441395', '0166274', '2114456', '0069954', '0423875', '0297082', '10227988', '0047926', '4118606', '1399193', '0188584', '19500164', '0021416', '0352638', '0476579', '9784864', '1180174', '1701202', '0164773', '16360458', '0100519', '6423362', '13310086', '0184929', '0413411', '0111940', '0028293', '0105323', '0092571', '0314519', '0114746', '13966962', '0367959', '0295967', '1586680', '1697623', '8106212', '0067819', '1075747', '12299824', '2155706', '0282985', '0213848', '0020492', '0758760', '0179813', '1661275', '0991064', '0263646', '5203980', '1822208', '0129222', '4016454', '0200200', '0016345', '3962850', '0050552', '0017929', '0307979', '27497758', '0077876', '0063039', '0043340', '8241768', '0306239', '0050905', '0239306', '0030026', '0046260', '1595374', '1070742', '0088814', '21146854', '4474320', '0010697', '0184085', '8076872', '0043495', '1958064', '0044718', '0252367', '0208247', '0119879', '15611020', '9778190', '8975220', '0063420', 

In [45]:
scraper.id_table.to_csv("incomplete_scraper_id_table_5-24.csv", index=False)

In [46]:
pd.read_csv("incomplete_scraper_id_table_5-24" + '.csv')

,id,title,keyword
0,5971474,The Little Mermaid,remake
1,7405458,A Man Called Otto,remake
2,6436620,White Men Can't Jump,remake
3,1586680,Shameless,remake
4,5635026,Peter Pan & Wendy,remake
...,...,...,...
43458,44284,OmnibusThe Remarkable Case of Mr. Bruhl (1952–...,based-on-short-story
43459,367296,The Book TowerBoy/Vampire Master (1979–1989,based-on-short-story
43460,7972136,Et lite grøss?Vertinnen (1973–1974,based-on-short-story
43461,9009076,Mesélö városokPrága (1975–1979,based-on-short-story


### Pulling Movie Info from Cinemagoer

In [ ]:
more_ids, more_ids_df = get_more_pages(["based-on-novel", "based-on-play"])

In [11]:
# GET CSV 
filename = 'example-file' # don't include '.csv'
scraper.export_csv(filename) 

In [ ]:
# READ THE CSV
df = pd.read_csv(filename + '.csv')
df

### Movie Scraper Class

In [14]:
%pip install git+https://github.com/cinemagoer/cinemagoer
%pip install --upgrade pip

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from imdb import Cinemagoer
import IPython.display as display
import pandas as pd
import time
import requests

ia = Cinemagoer()

In [34]:
class MovieScraper:
    def __init__(self, keywords, is_test):
        # Create list of IDs
        if (is_test):
            self.ids = self.get_test_ids(keywords)
        else:
            self.ids, self.id_table = self.get_full_ids(keywords)
        
    # Create a df using stored IDs
    def make_df(self):
        self.movies_df = self.create_dataframe()

    # Export dataframe from make_df with given filename
    def export_csv(self, filename):
        if self.movies_df is not None:
            filename += '.csv'
            self.movies_df.to_csv(filename, index=False)
        else:
            print("Error - Run make_df before exporting to CSV")

    # Collects every instance of the inputted keywords 
    def get_full_ids(self, keywords):
        search_id = [] # Stores all unique ids
        all_titles = [] # Stores all collected titles (including duplicates)
        all_keys = [] # Stores keywords in order corresponding to all_titles
        # Loop through every key word 
        for key in keywords:
            page_items = 1
            i = 1
            key_titles = [] # Stores titles collected for each keyword 
            # Iterate through every page in the keyword search
            while (page_items > 0):
                page_collect = ia.get_keyword(key, page=i)
                key_titles.extend(page_collect)
                page_items = len(page_collect)
                i += 1
                print(key.upper(), "- Retrieved Page", i-1)
                #time.sleep(.5) # Delay for visualization purposes (optional)
                display.clear_output(wait=True) # Clear console output

            # Add title, id, and keyword to lists (for later dataframe)
            all_titles.extend([str(title) for title in key_titles])
            search_id.extend([title.movieID for title in key_titles])
            all_keys.extend([key] * len(key_titles))
            
            # Report that the program has completed keyword search
            print(key.upper(), "- Collected", len(key_titles), "IDs over", i-1, "pages")
        
        # Creates dataframe of titles and ids collected for keywords (including duplicates)
        search_df = pd.DataFrame({'id': search_id, 'title': all_titles, 'keyword': all_keys})
        
        # Reports completion of overall search 
        search_id = [*set(search_id)] # Removes duplicate ids from overlaps in keywords
        print("Total:", len(search_id), "IDs collected - Removed", len(all_titles)-len(search_id), "duplicates")
        
        return search_id, search_df
    
    # Collects sample of keyword search results (first 50 items per keyword)
    def get_test_ids(self, keywords):
        search_id = []
        for key in keywords:    
            search = ia.get_keyword(key, page=1)
            search_id.extend([title.movieID for title in search])
            print(key.upper(), ": Collected", len(search_id), "IDs")
        og_length = len(search_id)
        search_id = [*set(search_id)] # Removes duplicate ids from overlaps in keywords
        print("Total:", len(search_id), "IDs collected - Removed", og_length-len(search_id), "duplicates")
        return search_id
    
    # Collect movie information from initialized ids 
    def create_dataframe(self):
        imdb_data = []
        error_counter = 0
        id_list = self.ids
        for id in id_list:
            try:
                movie_info = self.get_movie_info(id)
            except:
                error_counter += 1
                print('Error occurred:', ia.get_movie(id)['title'])
                movie_info = {'imdbID': None,
                    'title': None,
                    'production companies': None,
                    'director': None,
                    'producer': None,
                    'genres': None,
                    'keywords': None,
                    'rating': None,
                    'votes': None
                    }
            imdb_data.append(movie_info)

        # Remove empty rows from imdb_data
        imdb_data = [item for item in imdb_data if item is not None and item['imdbID'] is not None]
        # Report errors and removed
        print("Completed with", error_counter, "Errors and", (len(id_list) - len(imdb_data) - error_counter), "Removed Value(s)")
        df = pd.DataFrame(imdb_data)
        return df

    # HELPER - Returns information for media based on inputted ID if media is a movie. If not, returns None and prints that it was removed
    def get_movie_info(self, id):
        res = ia.get_movie(id)
        ia.update(res, 'keywords')
        
        if (res['kind'] == "movie"):
            return {'imdbID': id,
                'title': res.get('title'),
                'production companies': self.normalize_objects(res.get('production companies')),
                'director': self.normalize_objects(res.get('director')),
                'producer': self.normalize_objects(res.get('producer')),
                'genres': res.get('genres'),
                'keywords': res.get('keywords'),
                'rating': res.get('rating'),
                'votes': res.get('votes')
                }
        else:
            pass
            #print("Removed - Not a Movie: ", ia.get_movie(id)['title'])

    # HELPER - Method takes a list of Person or Company objects and returns it as a list of dicts with an ID and name
    def normalize_objects(self, objects):
        # Stores type value for IMDb Person object 
        test = ia.get_movie('0380510', ['main', 'vote details'])
        personType = type(test.get('producer')[0])
        
        new_list = []
        for item in objects:
            if type(item) is personType:
                id = item.personID
            else:
                id = item.companyID
            name = item.data['name']
            new_list.append({
                'id': id,
                'name': name
            })
        return new_list
    
    

In [40]:
# GET OTHER PAGES IN 10,000+ keyword values
def get_more_pages(keywords):
    search_id = [] # Stores all unique ids
    all_titles = [] # Stores all collected titles (including duplicates)
    all_keys = [] # Stores keywords in order corresponding to all_titles
    # Loop through every key word 
    for key in keywords:
        page_items = 1
        i = 201
        key_titles = [] # Stores titles collected for each keyword 
        # Iterate through every page in the keyword search
        while (page_items > 0):
            page_collect = ia.get_keyword(key, page=i)
            key_titles.extend(page_collect)
            page_items = len(page_collect)
            i += 1
            print(key.upper(), "- Retrieved Page", i-1)
            #time.sleep(.5) # Delay for visualization purposes (optional)
            #display.clear_output(wait=True) # Clear console output

        # Add title, id, and keyword to lists (for later dataframe)
        all_titles.extend([str(title) for title in key_titles])
        search_id.extend([title.movieID for title in key_titles])
        all_keys.extend([key] * len(key_titles))
        
        # Report that the program has completed keyword search
        print(key.upper(), "- Collected", len(key_titles), "IDs over", i-1, "pages")
    
    # Creates dataframe of titles and ids collected for keywords (including duplicates)
    search_df = pd.DataFrame({'id': search_id, 'title': all_titles, 'keyword': all_keys})
    
    # Reports completion of overall search 
    search_id = [*set(search_id)] # Removes duplicate ids from overlaps in keywords
    print("Total:", len(search_id), "IDs collected - Removed", len(all_titles)-len(search_id), "duplicates")
    
    return search_id, search_df